# SIXT33N Project
## Phase 1: Microphone Front End

### EE 16B: Designing Information Devices and Systems II, Fall 2018

Written by Nathaniel Mailoa and Emily Naviasky (2016)

nmailoa@berkeley.edu &emsp; enaviasky@berkeley.edu

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-


## Table of Contents

* [Introduction](#intro)
* [Part 1: Circuit Layout and Power](#part1)
* [Part 2: Microphone Biasing Circuit](#part2)
* [Part 3: Tuning the Microphone's Gain](#part3)
* [Part 4: Filtering](#part4)
* [Part 5: Data Collection](#part5)

<a id='intro'></a>
## Introduction

For the rest of this semester you will be designing SIXT33N, a mischevious little robot who *might* just do what you want - if you design it correctly. In this phase, **you will be desiging SIXT33N's ears**: a microphone front end circuit that processes the mic signal into something you can record using the Launchpad ADC.

The goals of this phase are as follows:
- Band pass filter circuit
- Level shift + gain circuit for ADC
- View ADC output on PC
- Record data for next phase
- Prepare to use batteries to power SIXT33N

<a id='part1'></a>
## <span style="color:blue">Part 1: Circuit Layout and Power</span>

### Materials
- Power supply + cables
- 1 LM7805 voltage regulator
- 2 1$\mu$F capacitors
- 1 10$\mu$F capacitor

In this part we will build the filtering circuit for our audio input to SIXT33N.

Our circuit will need two sources of power: 
1. ** 5V (Will come from the voltage regulator) **
2. ** 3.3V (Will come from the launchpad) **

** There is no negative voltage source. **

Try to divide your breadboard into the following sections so that you have room for them all.
- Use both of your (+) rails for 3.3V because most of the circuit will use 3.3V.
- Plug your 5V source into a row close to your mic board.  Today, only the mic board uses the 5V supply.
- Connect the ground of the 3.3V and 5V supplies, and connect the two (-) to create your ground rail.
- The purple section is where you will build your front end.
- The blue sections are set aside for your motor driver circuits later in the project

<img style="width:700px" src="images/circuit_outline_diagram_v2_with_voltage_reg_single_cap.png">

**KEEP YOUR CIRCUIT NEAT!** That goes without saying :)

#### Power Supply

** In the future: ** 
SIXT33N will run on a 9V battery when it is running around. 
- A voltage regulator will regulate this 9V down to 5V
- Our MSP will take in 5V from the voltage regulator and output 3.3V for our rails.  

** For today: ** Use the power supply at your bench to power your voltage regulator: 
- Power your voltage regulator with 9V from the bench power supply
- Note that you do not have a -5V rail, nor a -3.3V
- Connect both power supplies' grounds on your breadboard
- **Limit the current from your supply to 100mA (0.1 amperes)**

## Voltage Regulator

The Launchpad and the front end circuit need a 5V source, so we need a voltage regulator. The <a href="http://www.mouser.com/ds/2/308/LM7805-1124977.pdf">LM7805</a> is a voltage regulator that outputs 5V. Glance through the datasheet (figure 6) to find the pin diagram. 

**For now, connect $V_{\text{in}}$ to 9V from the power supply.** Don't forget to connect the ground of the regulator to the rail you want to be ground. 

<center>
<img width="500px" src="images/voltage_reg.png">
<img width="200px" src="images/voltage_reg_real.png">
<img width="550px" src="images/lmp.png">
</center>

**<span style="color:red">Use your multimeter to check that the output of your regulator is 5V.</span>**



Now, you are going to be working with some small signals. You don't want to introduce more noise than you need to, and the power rails can be a large source of noise. To keep some noise off of your power rails:

### **Add a decoupling capacitor (10 uF, code 106) between your 3.3 V rail and ground.**

Decoupling capacitors will reduce fluctuations on your rails (which can occur for example when the voltage source must supply a large amount of current across wires which actually have non-zero resistance and inductance) .  (https://en.wikipedia.org/wiki/Decoupling_capacitor) You should use them in every circuit you build <i>in the future for the rest of your life</i>.

<a id='part2'></a>
## <span style="color:blue">Part 2: Microphone Biasing Circuit</span>

### Materials
- 10k Resistors (x2)
- 51k Resistors (x2)
- 100k Resistor
- 50k Potentiometer
- Op-amps
- Mic Board

First, let's take a closer look at our mic board.  The following circuits are on the mic board PCB:


<img style="width:800px" src="./images/MicBoardSchematic_pot_overlay.png">

We will build circuits which provide signals for the OS1 (DC Offset) and OS2 (Level shift) pins.

#### Microphone Gain

The electret microphone (https://en.wikipedia.org/wiki/Electret_microphone) behaves as a ** variable current source ** depending on the size of the sound waves hitting it. Current is a bit difficult to work with in this case, so you will turn that into a voltage signal using the Mic Gain part of the circuit.

#### Buffer
This buffer helps keep the amplifier and the capacitor from affecting the microphone. It looks a little different than the buffers we usually use (there's a resistor in the feedback loop), but it functions just the same.

#### Remove Mic Drift

Putting a capacitor between one stage and the next is usually called a coupling cap (https://en.wikipedia.org/wiki/Capacitive_coupling), but it really just a high pass filter with a very low corner. The microphone naturally has a lot of low frequency drift, so we use this coupling cap to remove any DC offset and noise. This allows us to ignore whatever DC value the mic gain stage had, and add in a convenient DC value using the next stage.

#### DC Offset

For this project, **you don't have a negative power source**, only your 5V rail and ground (our LaunchPads don't like negative voltage as an input). If you center your signal around ground, like we did in previous labs, then you will lose the negative half the signal as soon as you send it through an op-amp because your op-amp won't be able to track your signal below ground. To get around this problem, we want to center our signal in the center of our voltage range.

However, you will have to <b>be careful</b> and remember that a DC offset exists or it could become troublesome. Think of a 0.1V DC signal. Now put that signal through a non-inverting op-amp with x100 gain. Suddenly that 0.1V DC signal becomes 10V! 

#### Level Shift

The OS2 Pin will allow us to introduce something called a level shift - it will help us deal with the problem of the DC offset and the amplifier.

#### Amplifier

Finally, the mic board uses a non-inverting amplifier to amplify the microphone signal. Note that OS2 is on the inverting terminal of the op-amp - we can use this to help us deal with our DC offset problem.


## Biasing Circuit

Looking at our mic board, we still need to determine the inputs to OS1 and OS2.

#### OS1: DC Offset
Because we need our signal to be centered in our usable range (0 - 3.3 V), we will need to set the DC offset to the midpoint of this range: 1.65 V. This can easily be accomplished with a voltage divider of two equal resistors from the 3.3 V rail to ground.

#### OS2: Level Shift
When we introduce the DC Offset, we will encounter a problem when the signal passes through the non-inverting amplifier: that DC offset will be amplified along with the rest of the signal! This is because the amplifier will amplify the signal as referenced from <i>ground</i>.

So, referencing from <i>ground</i> is the key problem here. If we want to avoid amplifying the DC offset, what value should we use for the reference? 

#### <span style="color:green"> If you guessed 1.65 V, congratulations! This is the key idea to a level-shifter.</span>

Recall that we connected OS2 to ground when we built color organ - this is how we told the non-inverting amplifier to use ground as a reference. To use 1.65 V instead, <b> we will need to connect OS2 to a non-zero voltage</b>. This voltage will need to match the DC offset we introduce to OS1.

However, we have another problem: our resistors can varry by up to 5%. This means it will be very unlikely that we can find two matched pairs of resistors so both OS1 and OS2 will be at exactly the same voltage.

To make biasing easier, you are using a new device. The resistor with an arrow through it is called a potentiometer. The potentiometer has 3 terminals. Think of the top two terminals as one resistor and the bottom two as a a second resistor. The two resistors must add up to 50k$\Omega$, but by turning the knob, you can make the first resistor large and the second small, or vice versa.

<img style="width:500px" src="images/pot.png">

<center><span style="color:black; font-weight: 400; font-size: 24px; font-style: normal"> Inside the potentiometer:</span></center>
<img style="width:300px" src="images/inside-potentiometer.jpg">

There are 2 ways potentiometers are usually used:
- Voltage divider (all 3 terminals used)
- Variable resistor (only top or bottom resistor used)

(In this lab we will use all 3 terminals.  The middle terminal will be connected to the non-inverting input of the op-amp)

**The figure below shows the two biasing circuits that we will need to build.** 

<img style="width:800px" src="images/BiasCircuit_51k_pot_overlay.png">

### Your tasks
- Build the above circuit
- Use 0 and +3.3V as the power supplies for these op-amps.
- Connect your mic board as follows:
    - VDD: 5 V from voltage regulator
    - VSS: ground
    - OS1 and OS2 to the biasing circuit we just built

In the next part, we will tune the mic board and biasing circuit by using a screwdriver to adjust the potentiometers.

<a id='part3'></a>
## <span style="color:blue">Part 3: Tuning the Microphone's Gain</span>

### Materials
- Oscilloscope + probe

When you first hook everything up, you might not get anything reasonable - this is totally okay! If the Level Shift (voltage at OS2) isn't *perfectly* matched to the DC offset, then the amplifier might amplify it to the point where the signal rails out!

Carefully tune both the <b>gain</b> and the <b>level shift</b> until the output signal is centered at 1.65 V. This can take a little bit of fiddling, so just be patient!

### Tuning Instructions
**Overview: Start with a low gain, tweak the level shift. Increase the gain a bit, tweak the level shift. Repeat until you're satisfied with your signal level.**

#### Setup:
1. Start by making sure your micboard is powered by $5\mathrm{V}$ from the voltage regulator at VDD and GROUNDED at VSS. 
2. Make sure OS1 is connected to the output of your voltage divider's buffer, and OS2 is connected to the output of your potentiometer's buffer. 
3. Connect an oscilloscope cable to the output.
4. Display a peak to peak measurement for the current channel on the oscilloscope
5. Display an average measurement for the current channel on the oscilloscope

#### Now you are ready to tune your micboard!
- In the project, you will be using your *voice*, not tones, with the mic board.
- While tuning, talk into the microphone from a comfortable distance (2-3 feet).
- This circuit will end up on your car on the ground, and you don't want to have your face on the ground to control it!


1. Lower the gain: Turn the potentiometer on your mic board **clockwise**.
2. Adjust the level shift: Turn the potentiometer on your breadboard until the average measurement is around $1.65\mathrm{V}$.
3. Increase the gain: Turn the gain potentiometer on the mic board **counter-clockwise** until the signal becomes uncentered.
4. Iterate on steps 2 and 3 until you see a signal with $\approx2.5\mathrm{V}_{pp}$ amplitude centered near $1.65\mathrm{V}$.

Congratulations! You have successfully tuned your micboard. 


#### Note:
 - The output voltage of the board ranges from $0$ to $3.3\mathrm{V}$ when the loudest sound is applied.
 - Test by speaking (loudly) from ~2 feet from the mic board.
 - We can always adjust this later.

Now, lets take some measurements of our mic board. Use your o-scope to probe at the OUTPUT pin.


**<span style="color:red">What is the DC voltage at the output?</span>**

YOUR ANSWER HERE

<a id='part4'></a>
## <span style="color:blue">Part 4: Filtering</span>

### Materials
- Op-amps
- Resistors
- Capacitors
- 1 Micboard + biasing circuit
- Power supply + cables
- Oscilloscope + probe

Recall from the previous week's lab that the frequency range for the human voice is about 180-20000Hz (female voice: 300 Hz-17kHz; male voice: 150Hz-8kHz), but most speech falls between **250Hz** and **2500Hz**. In order to minimize noise, we want to only collect data inside this frequency range. So, we'll build a bandpass filter like we did in the color organ lab. 

### Your tasks
- Calculate appropriate capacitor and resistor values for your high- and low-pass filters
- Build the following circuit on your breadboard
    - The buffer at the output with rails of 0V and 3.3V prevents damage to your LaunchPad.
    - The high-pass filter is connected to OS2 to keep the signal centered at 1.65V.
        - If it were connected to ground, the high-pass filter would remove the 1.65V offset.
- Use the micboard output as the input to your band-pass filter.

<center><img width='800px' src="./images/bandpass.png"></center>

**<span style="color:red">Record your choice of R1, R2, C1 and C2 for the project report!</span>**

In [2]:
low_pass_corner_freq = 200 #YOUR VALUE HERE

C1 = 1E-6 ohms #YOUR VALUE HERE
R1 = 800 #YOUR CODE HERE

high_pass_corner_freq = 2450 #YOUR VALUE HERE

C2 = 1E-6 #YOUR VALUE HERE
R2 = 65 #YOUR CODE HERE

print("Resistor for lowpass: ", R1)
print("Capacitor for lowpass: ", C1)

print("Resistor for highpass: ", R2)
print("Capacitor for highpass: ", C2)

Resistor for lowpass:  0
Capacitor for lowpass:  0
Resistor for highpass:  0
Capacitor for highpass:  0


Once you have built your circuit, check the following:

**<span style="color:red">Check that:</span>**
- The DC offset at each node is the same as your block diagram
- The gain of each amplifier is the same as your block diagram

**<span style="color:red">Check the frequency response of your circuit.</span>** Use the function generator to apply a specific input frequency with an offset of $1.65\mathrm{V}$ (instead of $\mathrm{V_{mic}}$). Watch the output of your signal on your scope and change the frequency from low to high. Check that the amplitude of the output at the cutoff frequency is attenuated as expected (i.e. check that your 3dB point happens at the cutoff frequency that you want).

<a id='part5'></a>
## <span style="color:blue">Part 5: Data Collection</span>

### Materials
- Part 1 setup
- Launchpad + USB cable

Your Launchpad will be colleting your speech data, so you will want to make sure that you can digitize your analog signal properly and nothing on your Launchpad is broken. 

- Upload <b>`adc_read.ino`</b> to the Launchpad.
- Connect pin `6.0` to the output of your circuit.
- Connect the ground of the Launchpad to the ground of your circuit.
- Press the reset button on your Launchpad.
- From the command line, run <b>`python adc_read.py -D COM###`</b> (substitute `###` with the proper number from Device Manager).

If you see a graph on your screen with a signal that changes when you make noises at your microphone and is centered in the middle of the graph, then you have succeeded! Congratulations!

<img width='30px' align='left' src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/check.png">
<br/>
## <span style="color:green">CHECKPOINT</span>

#### Checkoff Quiz: https://goo.gl/forms/l05O82RbLra0MReq2

 <span style="color:green">**Show your GSI your working circuit with the microphone.** Make sure the voltage ranges, gain and cutoff frequencies are as expected from your design. </span>

<span style="color:red">**Remember what each part of your circuit is for (Record this in a Google doc or somewhere else safe). You will need to write a summary for your final report.**</span>
